# 🔬Differentially Expressed Genes (DEG) Selection: Analyzing RNASeq data with DESeq2 package

This notebook finds the top differentially expressed genes (DEGs) using:
- ✅ DESeq2 with paired design in R

# Input files expected:
- `long_format_tumor_normal_samples.csv`: Sample metadata (Columns: SampleID, condition [Tumor/Normal], PatientID)
- `tcga_luad_raw_expression_data.csv`: RNA-seq matrix - raw fold-change values (rows = sample IDs, columns = genes). Downloaded from UCSC Xena public links for TCGA-LUAD: https://tcga.xenahubs.net/download/TCGA.LUAD.sampleMap/HiSeqV2.gz
- `expression_data_matched.csv`: RNA-seq data for samples which have matching methylation data (rows = sample IDs, columns = genes)

# Output produced:
- `DESeq2_paired_DEG_results.csv`: DESeq2 output (diffrencialy expressed gene (Columns: `baseMean`, `log2FoldChange`, `lfcSE`, `pvalue`, `padj`




##1) Setup & upload
###1.1) Getting the list of tumor/normal sampleId with both expression and methylation data

In [ ]:
# If running in Colab, use this to upload files. Otherwise ensure files exist in working directory.
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    print('Please upload long_format_tumor_normal_samples.csv, expression_data_matched.csv')
    uploaded = files.upload()
    print('Uploaded:', list(uploaded.keys()))

IN_COLAB = True
Please upload long_format_tumor_normal_samples.csv, expression_data_matched.csv


Saving expression_data_matched.csv to expression_data_matched.csv
Saving long_format_tumor_normal_samples.csv to long_format_tumor_normal_samples.csv
Uploaded: ['expression_data_matched.csv', 'long_format_tumor_normal_samples.csv']


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection

counts = pd.read_csv("expression_data_matched.csv", index_col=0)
meta = pd.read_csv("long_format_tumor_normal_samples.csv")

# Match paired samples with methylation data
tumor_samples = meta[meta["condition"] == "Tumor"]
normal_samples = meta[meta["condition"] == "Normal"]
paired = pd.merge(tumor_samples, normal_samples, on="PatientID", suffixes=("_Tumor", "_Normal")).dropna()
print('tumor/normal sample paired count which have methylation data:', len(paired))


# Transpose count matrix: genes as index
counts_T = counts.T

# Filter paired samples to keep only those present in counts_T columns
valid_tumor_samples = paired[paired["SampleID_Tumor"].isin(counts_T.columns)]
valid_paired = valid_tumor_samples[valid_tumor_samples["SampleID_Normal"].isin(counts_T.columns)]
print('tumor/normal sample paired count with both methylation & expression data:', len(valid_paired))
valid_paired.head()


tumor/normal sample paired count which have methylation data: 29
tumor/normal sample paired count with both methylation & expression data: 18


,SampleID_Tumor,condition_Tumor,PatientID,SampleID_Normal,condition_Normal
0,TCGA-44-6778-01,Tumor,TCGA-44-6778,TCGA-44-6778-11,Normal
1,TCGA-50-5931-01,Tumor,TCGA-50-5931,TCGA-50-5931-11,Normal
3,TCGA-44-2668-01,Tumor,TCGA-44-2668,TCGA-44-2668-11,Normal
4,TCGA-44-2665-01,Tumor,TCGA-44-2665,TCGA-44-2665-11,Normal
5,TCGA-44-6148-01,Tumor,TCGA-44-6148,TCGA-44-6148-11,Normal


### 1.2) Installing DESeq2 library

In [ ]:
!apt-get install -y r-base


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
r-base is already the newest version (4.5.1-1.2204.0).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
%%script R --vanilla
install.packages("BiocManager", repos="http://cran.us.r-project.org")
BiocManager::install("DESeq2", update=FALSE, ask=FALSE)


R version 4.5.1 (2025-06-13) -- "Great Square Root"
Copyright (C) 2025 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> install.packages("BiocManager", repos="http://cran.us.r-project.org")
> BiocManager::install("DESeq2", update=FALSE, ask=FALSE)
Creating a new generic function for ‘aperm’ in package ‘BiocGenerics’
Creating a new generic function for ‘append’ in package ‘BiocGenerics’
Creating a new generic function for ‘as.da

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/BiocManager_1.30.26.tar.gz'
Content type 'application/x-gzip' length 594489 bytes (580 KB)
downloaded 580 KB

* installing *source* package ‘BiocManager’ ...
** this is package ‘BiocManager’ version ‘1.30.26’
** package ‘BiocManager’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (BiocManager)

The downloaded source packages are in
	‘/tmp/RtmpB1sVAP/downloaded_packages’
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 

In [ ]:
!pip install -q rpy2
%load_ext rpy2.ipython

### 1.3) Upload raw RNAseq data - foldChange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    #Upload raw RNAseq data from google drive to feed to DESeq2
import pandas as pd
file_path = '/content/drive/My Drive/GenomicsProject_data/tcga_luad_raw_expression_data.csv'
counts_raw = pd.read_csv(file_path)

In [ ]:
# Preprocess counts_raw in Python to set SampleID as index and transpose
counts_preprocessed = counts_raw.set_index('Unnamed: 0')

# Preprocess meta to set SampleID as index
meta_processed = meta.set_index('SampleID')

# Display the head of the preprocessed data to verify
print("Preprocessed counts preview (SampleIDs as index, genes as columns):")
display(counts_preprocessed.head())
print("\nPreprocessed metadata preview (SampleIDs as index):")
display(meta_processed.head())

Preprocessed counts preview (SampleIDs as index, genes as columns):


,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,...,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS,base_id,sample_type
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-69-7978-01,9.9898,4.2598,0.4181,10.3657,11.1718,10.5897,12.2708,4.7670,0.0000,8.2023,...,6.2348,0.0000,9.4520,12.7565,8.2668,11.2400,6.1209,9.8977,TCGA-69-7978,Tumor
TCGA-62-8399-01,10.4257,11.6239,0.0000,11.5489,11.0200,9.2843,12.1540,5.7125,0.4628,5.5819,...,4.4464,1.3294,9.5226,12.2100,8.5437,10.3491,8.6398,9.7315,TCGA-62-8399,Tumor
TCGA-78-7539-01,9.6264,9.1362,1.1231,11.6692,10.4679,10.4649,12.6559,4.3943,0.3725,3.5365,...,6.0400,3.9201,9.2765,10.6498,6.1814,11.1659,6.0970,10.3540,TCGA-78-7539,Tumor
TCGA-50-5931-11,8.6835,9.4824,0.8221,11.7341,11.6787,11.5412,11.9285,5.9466,0.8221,3.3528,...,6.3782,0.0000,8.6781,14.6956,9.7151,10.5910,9.5115,10.4914,TCGA-50-5931,Normal
TCGA-73-4658-01,9.2078,5.0288,0.0000,11.6209,11.3414,10.9376,12.0539,6.0942,0.0000,7.4156,...,6.3898,1.1048,9.2697,13.0036,8.9786,10.6777,8.4187,10.3142,TCGA-73-4658,Tumor



Preprocessed metadata preview (SampleIDs as index):


,condition,PatientID
SampleID,,
TCGA-44-6778-01,Tumor,TCGA-44-6778
TCGA-50-5931-01,Tumor,TCGA-50-5931
TCGA-44-6144-01,Tumor,TCGA-44-6144
TCGA-44-2668-01,Tumor,TCGA-44-2668
TCGA-44-2665-01,Tumor,TCGA-44-2665


In [ ]:
# Filter the raw RNA-seq data file for tumor/normal samples with matching methylation and expression data
# valid_paired DataFrame has 'SampleID_Tumor' and 'SampleID_Normal' columns
valid_sample_ids = list(valid_paired['SampleID_Tumor']) + list(valid_paired['SampleID_Normal'])

# Filter counts_preprocessed and meta_processed to keep only the valid paired sample IDs
counts_filtered = counts_preprocessed.loc[valid_sample_ids]
meta_filtered = meta_processed.loc[valid_sample_ids]

# Display the head of the filtered data to verify
print("Filtered counts preview (SampleIDs as index, genes as columns):")
display(counts_filtered.head())
print("\nFiltered metadata preview (SampleIDs as index):")
display(meta_filtered.head())
print("\nNumber of samples after filtering:", counts_filtered.shape[0])


Filtered counts preview (SampleIDs as index, genes as columns):


,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,...,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS,base_id,sample_type
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-44-6778-01,9.6205,7.9642,1.5378,11.8432,11.0531,10.9005,12.4145,4.5366,2.0609,4.1805,...,6.1839,4.1291,8.9832,12.3412,9.0862,10.4779,9.4517,10.4395,TCGA-44-6778,Tumor
TCGA-50-5931-01,9.4932,7.6474,0.7060,12.0061,10.9403,10.2340,12.5008,3.1003,1.1780,9.6736,...,5.3270,1.8177,9.3925,11.2292,7.0270,9.7507,6.0857,9.6511,TCGA-50-5931,Tumor
TCGA-44-2668-01,9.4032,3.8726,0.0000,11.4589,11.1915,10.8651,11.3604,3.8454,0.0000,5.5501,...,5.0415,0.0000,9.8020,12.6297,8.0584,10.5011,6.5682,10.4597,TCGA-44-2668,Tumor
TCGA-44-2665-01,9.5904,6.6864,0.9626,11.5975,10.9343,10.9350,12.0246,3.7370,0.0000,8.7106,...,4.7343,0.9626,9.9923,12.8440,8.1411,10.9781,4.2499,10.4428,TCGA-44-2665,Tumor
TCGA-44-6148-01,8.9758,8.8888,0.6189,11.7523,11.0227,10.6612,12.5022,5.1410,0.0000,6.6141,...,4.3037,0.0000,9.6568,12.2590,7.1915,10.8171,7.5584,9.9243,TCGA-44-6148,Tumor



Filtered metadata preview (SampleIDs as index):


,condition,PatientID
SampleID,,
TCGA-44-6778-01,Tumor,TCGA-44-6778
TCGA-50-5931-01,Tumor,TCGA-50-5931
TCGA-44-2668-01,Tumor,TCGA-44-2668
TCGA-44-2665-01,Tumor,TCGA-44-2665
TCGA-44-6148-01,Tumor,TCGA-44-6148



Number of samples after filtering: 36


## 2) RNAseq analysis using DESeq2 library

In [ ]:
%%R -i counts_filtered -i meta_filtered -o res_df_r
library(DESeq2)
install.packages("ashr", repos="http://cran.us.r-project.org") # Install ashr package
library(ashr) # Load ashr library


# counts_filtered (SampleIDs as index, genes as columns) and
# meta_filtered (SampleIDs as index) dataframes are now available in R.

# Remove non-gene columns from counts_filtered before transposing
counts_filtered_genes <- counts_filtered[, !(colnames(counts_filtered) %in% c("base_id", "sample_type"))]


# Transpose the counts data to get genes as rows and samples as columns,
# which is the required format for DESeq2.
counts_raw_deseq2 <- t(as.matrix(counts_filtered_genes))


# Convert counts to a numeric matrix and ensure it's integer type
# Coerce errors to NA and then replace NAs with 0, as count data must be non-negative
counts_raw_deseq2 <- apply(counts_raw_deseq2, 2, function(x) as.numeric(as.character(x))) # Ensure numeric conversion
counts_raw_deseq2[is.na(counts_raw_deseq2)] <- 0
storage.mode(counts_raw_deseq2) <- "integer"
rownames(counts_raw_deseq2) <- colnames(counts_filtered_genes)

# Ensure metadata has SampleID as row names in R and is in the same order as counts columns
meta <- meta_filtered

# Ensure Condition and PatientID are factors with correct levels
meta$condition <- factor(meta$condition, levels=c("Normal", "Tumor"))
meta$PatientID <- factor(meta$PatientID)

# Remove genes with all zero counts after transposition and type conversion
counts_raw_deseq2 <- counts_raw_deseq2[rowSums(counts_raw_deseq2) > 0, ]

# Ensure metadata only contains samples present in the counts matrix after gene filtering
meta <- meta[colnames(counts_raw_deseq2), , drop = FALSE]

dds <- DESeqDataSetFromMatrix(countData=counts_raw_deseq2, colData=meta, design=~ PatientID + condition)
# Optional: filter out genes with low counts (e.g., sum of counts across all samples <= 10)
# Filtering can be done before creating the dds object or after
# dds <- dds[rowSums(counts(dds)) > 10, ]

dds <- DESeq(dds)

res <- results(dds, contrast=c("condition", "Tumor", "Normal"))
res <- lfcShrink(dds, coef="condition_Tumor_vs_Normal", type="ashr")

res_df_r <- as.data.frame(res)
rownames(res_df_r) <- rownames(res) # Explicitly set row names of the data frame to rownames of the results object
# The res_df_r dataframe will be passed back to the Python kernel
cat("DESeq2 analysis complete.")

Sample IDs from counts columns (after filtering): TCGA-44-6778-01 TCGA-50-5931-01 TCGA-44-2668-01 TCGA-44-2665-01 TCGA-44-6148-01 TCGA-50-5933-01 TCGA-44-6146-01 TCGA-50-5936-01 TCGA-49-6745-01 TCGA-50-5932-01 TCGA-44-6145-01 TCGA-44-6147-01 TCGA-50-5939-01 TCGA-44-5645-01 TCGA-38-4632-01 TCGA-50-5935-01 TCGA-50-5930-01 TCGA-73-4676-01 TCGA-44-6778-11 TCGA-50-5931-11 TCGA-44-2668-11 TCGA-44-2665-11 TCGA-44-6148-11 TCGA-50-5933-11 TCGA-44-6146-11 TCGA-50-5936-11 TCGA-49-6745-11 TCGA-50-5932-11 TCGA-44-6145-11 TCGA-44-6147-11 TCGA-50-5939-11 TCGA-44-5645-11 TCGA-38-4632-11 TCGA-50-5935-11 TCGA-50-5930-11 TCGA-73-4676-11 
Sample IDs from meta row names (after filtering): TCGA-44-6778-01 TCGA-50-5931-01 TCGA-44-2668-01 TCGA-44-2665-01 TCGA-44-6148-01 TCGA-50-5933-01 TCGA-44-6146-01 TCGA-50-5936-01 TCGA-49-6745-01 TCGA-50-5932-01 TCGA-44-6145-01 TCGA-44-6147-01 TCGA-50-5939-01 TCGA-44-5645-01 TCGA-38-4632-01 TCGA-50-5935-01 TCGA-50-5930-01 TCGA-73-4676-01 TCGA-44-6778-11 TCGA-50-5931-11 TCG

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/ashr_2.2-63.tar.gz'
Content type 'application/x-gzip' length 935117 bytes (913 KB)
downloaded 913 KB


The downloaded source packages are in
	‘/tmp/RtmpDUrCR5/downloaded_packages’
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
estimating size factors
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
estimating dispersions
gene-wise dispersion estimates
mean-di

## 3) Saving the top DEGs using fdr and fold_change values

In [ ]:
# Define significance thresholds (same as used before)
fdr_threshold = 0.05
log2fc_threshold = 1

# Filter the DESeq2 results DataFrame to get the DEGs
deseq2_degs_df = res_df_r[
    (res_df_r['padj'] < fdr_threshold) & (abs(res_df_r['log2FoldChange']) > log2fc_threshold)
]

# Save the filtered DataFrame to a CSV file
# Keep the index (gene names) as a column in the CSV
deseq2_degs_df.to_csv("DESeq2_paired_DEG_results.csv", index=True)

print("DESeq2 differentially expressed genes saved to DESeq2_paired_DEG_results.csv")

DESeq2 differentially expressed genes saved to DESeq2_paired_DEG_results.csv
